## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [1]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

### Sequential LCEL Chain

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate
                                    )

In [3]:
base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(
    base_url=base_url,
    model=model
)

In [4]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')
question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

template = ChatPromptTemplate.from_messages(messages=messages)

query = template.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})

response = llm.invoke(query)

print(response.content)

Here's an overview of the solar system in 5 points:

1. **The Sun is at center**: Our star, the Sun, is the central body of our solar system and provides light and heat to the planets.

2. **Mercury and Venus are close**: The two closest planets to the Sun, Mercury and Venus, have extremely high surface temperatures due to their proximity to it.

3. **Mars is a rocky planet**: Mars is known for its reddish appearance and has polar ice caps. Scientists believe there might have been water on Mars in the past, which could have supported life.

4. **Jupiter is the largest planet**: Jupiter is the largest planet in our solar system with massive storm systems like the Great Red Spot. It also has many moons orbiting around it.

5. **Earth and the other planets are unique**: Earth's atmosphere protects us from harm, while the planets on either side of the Sun (not including Earth) have conditions that could support life or be hostile to life as we know it.


##### Lets create a chain for above

In [5]:
chain = template | llm

In [6]:
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's what you need to know about the solar system:

• The Sun is at the center of our solar system.
• There are eight planets: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
• The four inner planets (Mercury, Venus, Mars, and Earth) are rocky and have air.
• The outer planets (Jupiter, Saturn, Uranus, and Neptune) are gas giants and have many moons.


In [8]:
response = chain.invoke({'school': 'PhD.', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's a brief overview of the solar system:

• The sun is at the center, and it's responsible for most solar energy on Earth.
• There are eight planets: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
• Pluto was once considered a planet but is now classified as a dwarf planet due to its small size.
• The four inner planets (Mercury, Mars, Venus, and Earth) orbit very close to the sun due to their rapid rotation periods.
• The outer planets (Jupiter, Saturn, Uranus, and Neptune) take much longer to complete one rotation around the sun due to their larger sizes.


##### StrOutputParser

In [7]:
chain = template | llm | StrOutputParser()

response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response)


Here's an overview of the solar system in 5 key points:

1. **The Sun**: At the center of our solar system is the star known as the Sun, which provides light and heat to the planets.

2. **The Inner Planets**: The four inner planets are Mercury, Venus, Earth, and Mars, which orbit closest to the Sun. These planets have no moons.

3. **Gas Giant**: The gas giant is Jupiter, a massive planet with storm systems that can be thousands of kilometers wide.

4. **Rings and Moons**: Saturn's ring system is one of the most prominent features in our solar system. Many moons orbit around Saturn, including Titan and Enceladus.

5. **The Outer Planets**: The four outer planets are Uranus and Neptune, which have very thin atmospheres compared to other planets in the solar system.


***

### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?

In [9]:
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='tell me about the {topics} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [17]:
analysis_prompt = ChatPromptTemplate.from_template(
    template="""Analyze the following text : {response}
    You need to tell me how difficult it is to understand the above text.
    Answer in one sentence only.
    """
)

analysis_prompt

ChatPromptTemplate(input_variables=['response'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['response'], input_types={}, partial_variables={}, template='Analyze the following text : {response}\n    You need to tell me how difficult it is to understand the above text.\n    Answer in one sentence only.\n    '), additional_kwargs={})])

In [14]:
analysis_chain = analysis_prompt | llm | StrOutputParser()

output = analysis_chain.invoke({'response': {response}})

print(output)

This text is relatively easy to understand as it uses basic definitions of terms such as "gas giant", "inner planets", and "outer planets" that are commonly known by students.


##### Combine the above 2 chains

In [15]:
combined_chain = {'response': chain} | analysis_prompt | llm | StrOutputParser()

output = combined_chain.invoke(
    {'school': 'primary', 
     'topics': 'solar system', 
     'points': 5}
)

print(output)

The text's simplicity and straightforward structure make it moderately easy to understand for a general audience, but its lack of depth and detailed information on specific planets and their characteristics may require some effort from those seeking more comprehensive knowledge.


In [16]:
combined_chain = {'response': chain} | analysis_prompt | llm | StrOutputParser()

output = combined_chain.invoke(
    {'school': 'PhD.', 
     'topics': 'solar system', 
     'points': 5}
)

print(output)

Understanding these 5 key points about the solar system requires a basic understanding of astronomy concepts such as planetary formation and celestial mechanics, but may still be challenging for those without prior knowledge or experience in the field.


***

### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.